# 第二十七讲 复数矩阵和快速傅里叶变换

## 复数矩阵

给定复数向量 $z = \begin{bmatrix}z_1 \\ z_2 \\ \dots \\ z_n\end{bmatrix}$，该向量不在实数空间 $\mathbb{R}^n$ 中，而是在复数空间 $\mathbb{C}^n$ 中。

复数矩阵的性质与实数矩阵的不同，一切计算基于共轭向量，$z = a + bi$，则它的共轭向量为 $z = a - bi$，并且共轭转置乘以原向量记为 $z^Hz$，$H$ 读作埃尔米特（人名为Hermite，形容词为Hermitian）：
* 模计算：$\begin{vmatrix}z\end{vmatrix}^2 = \bar{z}^Tz$，即 $z^Hz$。例，$z = \begin{bmatrix}1 \\ i \end{bmatrix}$，$\begin{vmatrix}z\end{vmatrix}^2 = \bar{z}^Tz = 1 + 1 = 2$。
* 内积计算：同模计算，有 $\bar{y}^Tx$，即 $y^Hx$。
* 对称性：对于实矩阵，$A^T = A$ 即可表达矩阵的对称性，而对于复矩阵则是 $\bar{A}^T = A$，即 $bar{A}^H$。
* 正交性：$q_i^Hq_j = \left\{\begin{matrix}0 & i≠j \\ 1 & i=j  \end{matrix}\right.$，对于正交矩阵则有 $Q^HQ = I$，并且对于复矩阵的正交矩阵，我们称为酉矩阵（Unitary Matrix）。

## 傅里叶矩阵

傅里叶矩阵（Fourier Matrix）的形式为 $F_n = \begin{bmatrix} 1 & 1 & 1 & \dots & w^{n-1} \\1 & w & w^2 & \dots & w^{2(n-1)} \\1 & w^2 & w^4 & \dots & w^{3(n-1)} \\\vdots &\vdots & \vdots & \ddots  & \vdots \\1 & w^{n-1} & w^{2(n-1)} & \dots & w^{(n-1)(n-1)}\end{bmatrix}$，其中 $(F_n)_{ij}=w^{ij}$，而 $w$ 满足 $w^n = 1$，即 $w = e^{i2\pi/n}$。

例如，当 $n=4$ 时，$w = e^{i\pi/2}$，并且有 $w = 1, w^2 = -1, w_3 = -i, w_4 = 1$，$F_4 = \begin{bmatrix}1 & 1 & 1 & 1 \\ 1 & i & i^2 & i^3 \\ 1 & i^2 & i^4 & i^6 \\ 1 & i^3 & i^6 & i^9\end{bmatrix} = \begin{bmatrix}1 & 1 & 1 & 1 \\ 1 & i & -1 & -i \\ 1 & -1 & 1 & -1 \\ 1 & -i & -1 & i \end{bmatrix}$。此时的 $F_4$ 并不是标准的酉矩阵，我们可以令 $F_4 = \frac{1}{2} \begin{bmatrix}1 & 1 & 1 & 1 \\ 1 & i & -1 & -i \\ 1 & -1 & 1 & -1 \\ 1 & -i & -1 & i \end{bmatrix}$（因为矩阵中向量的模为 $2$），这样就符合 $F_4^HF_4 = I$。

## 快速傅里叶变换

快速傅里叶变换（Fast Fourier Transform，FFT）可以减少矩阵乘法的计算量，在一般情况下，一个 $n$ 阶矩阵乘以一个列向量的复杂度为 $o(n^2)$，而通过快速傅里叶变换算法，可以将复杂度优化到 $o(nlogn)$。 算法的核心是通过大小倍数之间的关系，例如 $F_{64}$ 和 $F_{32}$。

这里通过分解 $F_{64}$ 来理解算法的过程：

首先 $F_{64} = \begin{bmatrix}I & D \\ I & -D \end{bmatrix}\begin{bmatrix}F_{32} & 0 \\ 0 & F_{32}\end{bmatrix}\begin{bmatrix} 1 & 0 & \dots & 0 & 0 & 0 & \dots & 0 \\  0 & 0 & \dots & 0 & 1 & 0 & \dots & 0 \\  0 & 1 & \dots & 0 & 0 & 0 & \dots & 0 \\ 0 & 0 & \dots & 0 & 0 & 1 & \dots & 0 \\ \vdots & \vdots & \ddots & \vdots & \vdots & \vdots & \ddots & \vdots \\ \vdots & \vdots & \ddots & \vdots & \vdots & \vdots & \ddots & \vdots \\ 0 & 0 & \dots & 1 & 0 & 0 & \dots & 0\\ 0 & 0 & \dots & 0 & 0 & 0 & \dots & 1 \end{bmatrix}$。

这里分开考虑三个矩阵：
* 第一个矩阵有单位矩阵 $I$ 和对角矩阵 $D = \begin{bmatrix}1 & 0 & 0 & \dots & 0 \\ 0 & w & 0 & \dots & 0 \\ 0 & 0 & w^2 & \dots & 0 \\ \vdots & \vdots & \vdots & \ddots & \vdots \\ 0 & 0 & 0 & \dots & w^{n-1}\end{bmatrix}$，该矩阵称为修正矩阵，单位矩阵的计算量可以忽略，对角矩阵的计算量为 $32$。
* 第二个矩阵为两个 $F_{32}$ 组成，计算量为 $2 * 32^2$。
* 第三个矩阵为一个置换矩阵，记作 $P$，作用是将奇数列置于偶数列前面，即将 $\begin{bmatrix} c_0 & c_1 & \dots\end{bmatrix}$ 变为 $\begin{bmatrix} c_0 & c_2 & \dots & c_1 & c_3 & \dots \end{bmatrix}$，该计算量也可忽略。

对于 $F_{32}$ 我们可以继续分解为 $F_{16}$，所以总的计算量满足 $nlogn$。